# Digits recognition

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import tensorflow.compat.v2 as tf
import mnist
import tensorflow.compat.v2.keras.utils as tf_utils
from tensorflow.compat.v2.keras.models import Sequential
from tensorflow.compat.v2.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, BatchNormalization
from tensorflow.compat.v2.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.compat.v2.keras.preprocessing.image import ImageDataGenerator

In [2]:
x_train = np.load("x.npy")
y_train = np.load("y.npy")

In [3]:
mn = mnist.load_data()

In [4]:
x_test = np.concatenate((mn[0][0], mn[1][0]))
y_test = np.concatenate((mn[0][1], mn[1][1]))

In [5]:
x_train = x_train.reshape((x_train.shape[0], 28, 28, 1)).astype('float32')
x_test = x_test.reshape((x_test.shape[0], 28, 28, 1)).astype('float32')

In [6]:
x_train = x_train / 255
x_test = x_test / 255

In [7]:
y_train = tf_utils.to_categorical(y_train)
y_test = tf_utils.to_categorical(y_test)

In [8]:
def make_model():
    model = Sequential()
    model.add(Conv2D(filters=48, kernel_size=5, input_shape=(28, 28, 1), activation='relu'))
    model.add(MaxPooling2D(pool_size=2, strides=1))
    model.add(BatchNormalization())
    model.add(Conv2D(filters=48, kernel_size=3, activation='relu'))
    model.add(MaxPooling2D(pool_size=2))
    model.add(BatchNormalization())

    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))
    model.add(Dense(128, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))

    model.add(Dense(10, activation='softmax'))
    
    model.compile(loss='categorical_crossentropy', optimizer='nadam', metrics=['accuracy'])
    return model

In [9]:
model = make_model()

early_stopping = EarlyStopping(monitor='accuracy', patience=10, restore_best_weights=True)
lr_schedule = ReduceLROnPlateau(monitor='accuracy', factor=0.3, patience=6)

# try out featurewise_center=True 
generator = ImageDataGenerator(rotation_range=20,
                               width_shift_range=0.1,
                               height_shift_range=0.1,
                               zoom_range=0.1)

model.fit(generator.flow(x_train, y_train, batch_size=64),
          validation_data=(x_test, y_test), batch_size=64,
          epochs=10, verbose=2, steps_per_epoch = len(x_train) / 64,
          callbacks=[early_stopping, lr_schedule])

scores = model.evaluate(x_test, y_test)

print("CNN accuracy: ", scores[1])

Epoch 1/10
782/781 - 109s - loss: 0.2779 - accuracy: 0.9142 - val_loss: 0.0873 - val_accuracy: 0.9726
Epoch 2/10
782/781 - 110s - loss: 0.1208 - accuracy: 0.9634 - val_loss: 0.0538 - val_accuracy: 0.9835
Epoch 3/10
782/781 - 146s - loss: 0.0958 - accuracy: 0.9704 - val_loss: 0.0593 - val_accuracy: 0.9818
Epoch 4/10
782/781 - 124s - loss: 0.0850 - accuracy: 0.9740 - val_loss: 0.0555 - val_accuracy: 0.9830
Epoch 5/10
782/781 - 100s - loss: 0.0767 - accuracy: 0.9769 - val_loss: 0.0343 - val_accuracy: 0.9894
Epoch 6/10
782/781 - 100s - loss: 0.0715 - accuracy: 0.9783 - val_loss: 0.0291 - val_accuracy: 0.9914
Epoch 7/10
782/781 - 100s - loss: 0.0647 - accuracy: 0.9797 - val_loss: 0.0313 - val_accuracy: 0.9901
Epoch 8/10
782/781 - 100s - loss: 0.0602 - accuracy: 0.9821 - val_loss: 0.0234 - val_accuracy: 0.9928
Epoch 9/10
782/781 - 101s - loss: 0.0594 - accuracy: 0.9820 - val_loss: 0.0216 - val_accuracy: 0.9934
Epoch 10/10
782/781 - 103s - loss: 0.0571 - accuracy: 0.9827 - val_loss: 0.0196 - 

In [11]:
model.save('model.h5')